In [ ]:
import gymnasium as gym
import src.gymnasium_env


def query_environment(name):
    env = gym.make(name)
    spec = gym.spec(name)
    print(f"Action Space: {env.action_space}")
    print(f"Observation Space: {env.observation_space}")
    print(f"Max Episode Steps: {spec.max_episode_steps}")
    print(f"Nondeterministic: {spec.nondeterministic}")
    print(f"Reward Threshold: {spec.reward_threshold}")


query_environment("GridWorld-v0")

# observation, info = env.reset(seed=42)

Action Space: Discrete(4)
Observation Space: Dict('agent': Box(0, 9, (2,), int64), 'goal': Box(0, 9, (2,), int64))
Max Episode Steps: None
Nondeterministic: False
Reward Threshold: None


In [66]:
from tqdm.notebook import tqdm

bisogna fare una funzione chiamata `train_dqn_agent`, in cui:
- la funzione riceve due parametri `num_episodes` e `grid_size`
- inizialmente istanzia l'`env` e l'`agent`
- ad ogni episodio resetta l'`env`
- per ogni episodio fino che non ha finito (stato terminale):
    - l'`agent` seleziona un azione tramite epsilon greedy
    - l'`agent` ha la seguente funzione `step(s,a,r,s',a')` che viene chiamata per calcolare la loss, calcolare il gradiente e aggiornare la `q_network`

In [ ]:
import jax
import jax.numpy as jnp
from jax import random

# random consumes the key but not modify it
key = random.key(42)
print(key)

Array((), dtype=key<fry>) overlaying:
[ 0 42]


In [ ]:
for i in range(3):
    key, subkey = random.split(key)
    val = random.normal(subkey)
    print(f"draw {i}: {val}")

draw 0: 0.6057640314102173
draw 1: -0.21089035272598267
draw 2: -0.3948981463909149


In [ ]:
# no sequential guarantee
import numpy as np

key = random.key(42)
subkeys = random.split(key, 6)
print(np.stack([random.normal(subkey) for subkey in subkeys]))
print("vectorized:", jax.vmap(random.normal)(subkeys))

[ 0.07592554  0.60576403  0.4323065  -0.2818947   0.6549178  -0.2166012 ]
vectorized: [ 0.07592554  0.60576403  0.4323065  -0.2818947   0.6549178  -0.2166012 ]


In [82]:
!pip install loguru

/home/aarcara/miniconda3/envs/rl/lib/python3.11/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Using cached loguru-0.7.3-py3-none-any.whl.metadata (22 kB)
Using cached loguru-0.7.3-py3-none-any.whl (61 kB)


In [ ]:
from flax import nnx
import optax
from loguru import logger
from src.q_network import QNetwork


rngs = nnx.Rngs(0)
model = QNetwork(4, 128, 4, rngs=rngs)
optimizer = nnx.Optimizer(model, optax.adam(1e-3))

dummy_state = jnp.ones((4,), dtype=jnp.float32)


# da sistemare
def loss_fn(model: QNetwork, batch):
    err = model(batch)
    return jnp.mean(jnp.square(err))


@nnx.jit
def train_step(optim: nnx.Optimizer, batch):
    loss, grads = nnx.value_and_grad(loss_fn)(optim.model, batch)
    optim.update(grads)


class DeepQLearningAgent:
    def __init__(self, state_dim: int, action_dim: int, hidden_dim: int = 128, eps_start: float = 1.0):
        self.eps = eps_start
        q_network = QNetwork(state_dim, hidden_dim, action_dim, rngs=nnx.Rngs(42))
        tx = optax.adam(1e-3)
        self.state = nnx.Optimizer(q_network, tx)

    def act(self) -> int:
        # da sistemare
        if random.random() < self.eps:
            return np.argmax(self.Q)
        else:
            #q_vals = self.optim.model(jnp.)
            return random.randint(0, len(self.Q) - 1)

    def learn(self):
        # da implementare
        pass


def train_dql_agent(num_episodes: int = 500, grid_size: int = 10):
    env = gym.make("GridWorld-v0", size=grid_size)
    state_dim  = (
        env.observation_space["agent"].shape[0] +
        env.observation_space["goal"].shape[0]
    )
    action_dim = env.action_space.n
    logger.debug("State dim: {}, Action dim: {}", state_dim, action_dim)
    agent = DeepQLearningAgent(state_dim, action_dim)

    for _ in tqdm(range(num_episodes)):
        state, _ = env.reset()
        #state = preprocess(obs)

        done = False
        while not done:
            #action = agent.act()
            action = env.action_space.sample()
            next_state, reward, done, _, _ = env.step(action)

            state = next_state


#train_dql_agent()


# agent = DeepQLearningAgent()
# agent.train_step(jnp.ones((3,)))


[ 0.3933988  -0.3365822   0.14810538 -0.15271175]
